In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import statistics as stat
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import transformers
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup, AutoTokenizer, DebertaForSequenceClassification, DebertaModel
import torch
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import accuracy_score
import pickle as pkl
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, random_split
import string

In [3]:
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [7]:
urw_rec = {
"Blaming the war on others rather than the invader": ["Ukraine is the aggressor", "The West are the aggressors"],
"Discrediting Ukraine": ["Rewriting Ukraine’s history", "Discrediting Ukrainian nation and society", "Discrediting Ukrainian military", "Discrediting Ukrainian government and officials and policies", "Ukraine is a puppet of the West", "Ukraine is a hub for criminal activities", "Ukraine is associated with nazism", "Situation in Ukraine is hopeless"],
"Russia is the Victim": ["The West is russophobic", "Russia actions in Ukraine are only self-defence", "UA is anti-RU extremists"],
"Praise of Russia": ["Praise of Russian military might", "Praise of Russian President Vladimir Putin", "Russia is a guarantor of peace and prosperity", "Russia has international support from a number of countries and people", "Russian invasion has strong national support"],
"Overpraising the West": ["NATO will destroy Russia", "The West belongs in the right side of history", "The West has the strongest international support"],
"Speculating war outcomes": ["Russian army is collapsing", "Russian army will lose all the occupied territories", "Ukrainian army is collapsing"],
"Discrediting the West, Diplomacy": ["The EU is divided", "The West is weak", "The West is overreacting", "The West does not care about Ukraine, only about its interests", "Diplomacy does/will not work", "West is tired of Ukraine"],
"Negative Consequences for the West": ["Sanctions imposed by Western countries will backfire", "The conflict will increase the Ukrainian refugee flows to Europe"],
"Distrust towards Media": ["Western media is an instrument of propaganda", "Ukrainian media cannot be trusted"],
"Amplifying war-related fears": ["By continuing the war we risk WWIII", "Russia will also attack other countries", "There is a real possibility that nuclear weapons will be employed", "NATO should/will directly intervene"],
"Hidden plots by secret schemes of powerful groups": []}

In [8]:
cc_rec = {
"Criticism of climate policies" : ["Climate policies are ineffective", "Climate policies have negative impact on the economy", "Climate policies are only for profit"],
"Criticism of institutions and authorities": ["Criticism of the EU", "Criticism of international entities", "Criticism of national governments", "Criticism of political organizations and figures"],
"Climate change is beneficial": ["CO2 is beneficial", "Temperature increase is beneficial"],
"Downplaying climate change": ["Climate cycles are natural", "Weather suggests the trend is global cooling", "Temperature increase does not have significant impact", "CO2 concentrations are too small to have an impact", "Human activities do not impact climate change", "Ice is not melting", "Sea levels are not rising", "Humans and nature will adapt to the changes"],
"Questioning the measurements and science": ["Methodologies/metrics used are unreliable/faulty", "Data shows no temperature increase", "Greenhouse effect/carbon dioxide do not drive climate change", "Scientific community is unreliable"],
"Criticism of climate movement": ["Climate movement is alarmist", "Climate movement is corrupt", "Ad hominem attacks on key activists"],
"Controversy about green technologies": ["Renewable energy is dangerous", "Renewable energy is unreliable", "Renewable energy is costly", "Nuclear energy is not climate friendly"],
"Hidden plots by secret schemes of powerful groups": ["Blaming global elites", "Climate agenda has hidden motives"],
"Amplifying Climate Fears": ["Earth will be uninhabitable soon", "Amplifying existing fears of global warming", "Doomsday scenarios for humans", "Whatever we do it is already too late"],
"Green policies are geopolitical instruments": ["Climate-related international relations are abusive/exploitative", "Green activities are a form of neo-colonialism"]}

In [9]:
from collections import defaultdict

label2Id = {}
Id2Label = {}
count = 0

dominantLabel2Id = {}
dominantId2Label = {}
dominantCount = 0

label2Narr = [0 for i in range(96)]

for k in urw_rec.keys():

  dominantLabel2Id["URW: "+ k] = dominantCount
  dominantId2Label[dominantCount] = "URW: "+ k
  dominantCount += 1

  urw_rec[k].append("Other")
  for i in urw_rec[k]:
    label = "URW: " + k + ": " + i
    label2Id[label] = count
    Id2Label[count] = label

    label2Narr[count] = dominantCount - 1
    count += 1

for k in cc_rec.keys():

  dominantLabel2Id["CC: "+ k] = dominantCount
  dominantId2Label[dominantCount] = "CC: "+ k
  dominantCount += 1

  cc_rec[k].append("Other")
  for i in cc_rec[k]:
    label = "CC: " + k + ": " + i
    label2Id[label] = count
    Id2Label[count] = label

    label2Narr[count] = dominantCount - 1
    count += 1

label2Id["Other"] = count
Id2Label[count] = "Other"

dominantLabel2Id["Other"] = dominantCount
dominantId2Label[dominantCount] = "Other"

label2Narr[count] = dominantCount

In [10]:
label2Id

{'URW: Blaming the war on others rather than the invader: Ukraine is the aggressor': 0,
 'URW: Blaming the war on others rather than the invader: The West are the aggressors': 1,
 'URW: Blaming the war on others rather than the invader: Other': 2,
 'URW: Discrediting Ukraine: Rewriting Ukraine’s history': 3,
 'URW: Discrediting Ukraine: Discrediting Ukrainian nation and society': 4,
 'URW: Discrediting Ukraine: Discrediting Ukrainian military': 5,
 'URW: Discrediting Ukraine: Discrediting Ukrainian government and officials and policies': 6,
 'URW: Discrediting Ukraine: Ukraine is a puppet of the West': 7,
 'URW: Discrediting Ukraine: Ukraine is a hub for criminal activities': 8,
 'URW: Discrediting Ukraine: Ukraine is associated with nazism': 9,
 'URW: Discrediting Ukraine: Situation in Ukraine is hopeless': 10,
 'URW: Discrediting Ukraine: Other': 11,
 'URW: Russia is the Victim: The West is russophobic': 12,
 'URW: Russia is the Victim: Russia actions in Ukraine are only self-defence

In [11]:
enRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/EN/raw-documents/"
enSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/EN/subtask-2-annotations.txt"

hiRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/HI/raw-documents/"
hiSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/HI/subtask-2-annotations.txt"

bgRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/BG/raw-documents/"
bgSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/BG/subtask-2-annotations.txt"

ptRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/PT/raw-documents/"
ptSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/PT/subtask-2-annotations.txt"

ruRawFiles = "drive/MyDrive/SemEvalTaskData/target_4_December_release/RU/raw-documents/"
ruSubTask2File = "drive/MyDrive/SemEvalTaskData/target_4_December_release/RU/subtask-2-annotations.txt"

files_of_interest = "drive/MyDrive/SemEvalTaskData/target_4_December_release/files_of_interest.txt"

In [12]:
files = open(files_of_interest, "r").read().split("\n")
files.pop()

''

In [13]:
class NewsDatasetMultilingual(Dataset):
  def __init__(self):
    self.files = files
    # self.annotation_file = annotations_file
    self.enSubTask2File = enSubTask2File
    self.hiSubTask2File = hiSubTask2File
    self.bgSubTask2File = bgSubTask2File
    self.ptSubTask2File = ptSubTask2File
    self.ruSubTask2File = ruSubTask2File
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index: int):
    file_name  = self.files[index]
    text = open(file_name, "r", encoding='utf-8').read()
    text_labels = []
    narr_text_labels = []
    total_labels = [0 for i in range(96)]
    narr_labels = [0 for i in range(22)]

    sub_narrative_df = {}
    narrative_df = {}

    for k in label2Id.keys():
      sub_narrative_df[k] = 0

    for k in dominantLabel2Id.keys():
      narrative_df[k] = 0

    encoded_input = self.tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

    if "EN" in file_name:
      annotation_file = self.enSubTask2File
    elif "HI" in file_name:
      annotation_file = self.hiSubTask2File
    elif "BG" in file_name:
      annotation_file = self.bgSubTask2File
    elif "PT" in file_name:
      annotation_file = self.ptSubTask2File
    elif "RU" in file_name:
      annotation_file = self.ruSubTask2File

    with open(annotation_file, encoding='utf-8') as annotations:
      for line in annotations:
        splits = line.split("\t")

        if file_name.split("/")[-1] == splits[0]:
          sub_narrative_df['fileName'] = splits[0]
          narrative_df['fileName'] = splits[0]
          text_labels = splits[2].replace("\n", "").split(';')
          narr_text_labels = splits[1].split(';')
          break

    is_CC = 0
    is_URW = 0

    for l in text_labels:
      total_labels[label2Id[l]] = 1
      sub_narrative_df[l] = 1

    for l in narr_text_labels:
      if dominantLabel2Id[l] <= 10:
        is_URW = 1
      elif dominantLabel2Id[l] <= 20:
        is_CC = 1

      narr_labels[dominantLabel2Id[l]] = 1
      narrative_df[l] = 1


    rec = {}

    rec = {
        **encoded_input,
        "narr_labels": torch.FloatTensor(narr_labels),
        "sub_narr_labels": torch.FloatTensor(total_labels),
        "narr_dic": narrative_df,
        "sub_narr_dic": sub_narrative_df,
        "fileName": file_name.split("/")[-1],
        "is_CC": is_CC,
        "is_URW": is_URW,
        "urw_cc_labels": torch.FloatTensor([is_URW, is_CC])
    }

    return rec

In [14]:
total_dataset = NewsDatasetMultilingual()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
total_dataset[200]

{'input_ids': tensor([[  101, 12548, 10103,   163,   119,   161,   119, 10743, 17260, 10104,
          19877, 13208, 15833, 10114, 17115, 10313, 18507, 10114, 10103, 15094,
            131, 12548, 10103,   163,   119,   161,   119, 10743, 17260, 10104,
          19877, 13208, 15833, 10114, 17115, 10313, 10114, 10103, 15094,   131,
            151, 18593, 10103, 18507, 10114, 10103, 15094,   117, 10110, 10768,
          10127,   143, 20674, 34324, 10986, 12671, 10488, 21506, 10103,   163,
            119,   161,   119, 10110, 10103, 10873, 16613, 26483, 14935, 14693,
          10173, 29506, 10103, 13353, 37910, 74815,   117, 10362, 21291, 10203,
            100,   161, 14666, 22430, 10114, 15126, 10144, 27137, 11101, 10104,
          10103, 10313,   119, 11222, 11260, 11227, 21666, 22340, 10107, 10986,
          16015, 10110, 13726, 14176, 16825, 30959,   117, 10768, 10127, 10399,
          12495, 20674, 22588,   131, 30959, 10438, 21918, 10165,   119, 48673,
            117, 11222, 171

In [15]:
#student network
class BERTNarrativeClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTNarrativeClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        return logits

In [ ]:
dominantLabel2Id

{'URW: Blaming the war on others rather than the invader': 0,
 'URW: Discrediting Ukraine': 1,
 'URW: Russia is the Victim': 2,
 'URW: Praise of Russia': 3,
 'URW: Overpraising the West': 4,
 'URW: Speculating war outcomes': 5,
 'URW: Discrediting the West, Diplomacy': 6,
 'URW: Negative Consequences for the West': 7,
 'URW: Distrust towards Media': 8,
 'URW: Amplifying war-related fears': 9,
 'URW: Hidden plots by secret schemes of powerful groups': 10,
 'CC: Criticism of climate policies': 11,
 'CC: Criticism of institutions and authorities': 12,
 'CC: Climate change is beneficial': 13,
 'CC: Downplaying climate change': 14,
 'CC: Questioning the measurements and science': 15,
 'CC: Criticism of climate movement': 16,
 'CC: Controversy about green technologies': 17,
 'CC: Hidden plots by secret schemes of powerful groups': 18,
 'CC: Amplifying Climate Fears': 19,
 'CC: Green policies are geopolitical instruments': 20,
 'Other': 21}

In [ ]:
temp = torch.tensor([[2, 3, 4], [5, 6, 7]])

In [ ]:
temp = torch.tensor([2, 3, 4])
temp2 = torch.tensor([5, 6, 7])

2 - temp - temp2

tensor([-5, -7, -9])

In [ ]:
temp[:,dominantLabel2Id['URW: Discrediting Ukraine']].unsqueeze(1).shape

torch.Size([2, 1])

In [ ]:
#rule projection of the
class Rule:
  def __init__(self, index1, index2, numClasses):
    print("index1", index1, "index2", index2)
    self.index1 = index1
    self.index2 = index2
    self.numClasses = numClasses

  def calculate_rule(self, logits):
    updated_probs = torch.zeros_like(logits)
    prob_index1 = nn.Sigmoid()(logits[:, self.index1])
    prob_index2 = nn.Sigmoid()(logits[:, self.index2])

    # print("prob_index1", prob_index1)
    # print("prob_index2", prob_index2)
    rule_eval = 1 - torch.minimum(2 - prob_index1 - prob_index2, torch.ones_like(prob_index1))
    # print("rule_eval", rule_eval.shape, "updated_probs", updated_probs.shape)
    updated_probs[:, self.index1] = rule_eval
    updated_probs[:, self.index2] = rule_eval

    return updated_probs

class RuleEvaluation:
  def __init__(self, num_labels, rules, regularization_terms, confidence):
    self.rules = rules
    self.num_labels = num_labels
    self.regularization_terms = regularization_terms
    self.confidence = confidence

  def calculate_rules(self, logits):
    total_rules = torch.zeros_like(logits)

    for i in range(len(self.rules)):
      total_rules = total_rules + self.rules[i].calculate_rule(logits) * self.regularization_terms[i]

    # print("total_rules shape", total_rules.shape)
    # print("total_rules", total_rules)

    return torch.exp( -1 * total_rules * self.confidence)

In [ ]:
temp1 = torch.tensor([[2, 3, 4], [5, 6, 7]])

temp2 = torch.tensor([[2, 3, 4], [5, 6, 7]])

temp1 * temp2

tensor([[ 4,  9, 16],
        [25, 36, 49]])

In [ ]:
temp1[:,1] = update

temp1

tensor([[ 2, 10,  4],
        [ 5, 12,  7]])

In [20]:
class TeacherStudentNarrativeClassifier(nn.Module):
  def __init__(self, num_labels):
    super(TeacherStudentNarrativeClassifier, self).__init__()

    # rule1 = Rule(dominantLabel2Id['URW: Blaming the war on others rather than the invader'], dominantLabel2Id['URW: Overpraising the West'], num_labels)
    rule2 = Rule(dominantLabel2Id['URW: Overpraising the West'], dominantLabel2Id['URW: Distrust towards Media'], num_labels)

    # rule3 = Rule(dominantLabel2Id['CC: Climate change is beneficial'], dominantLabel2Id['CC: Amplifying Climate Fears'], num_labels)
    rule4 = Rule(dominantLabel2Id['CC: Downplaying climate change'], dominantLabel2Id['CC: Amplifying Climate Fears'], num_labels)

    self.student = BERTNarrativeClassifier(num_labels)
    self.rule_evaluation = RuleEvaluation(num_labels, [ rule2, rule4], [100, 100], 100)

  def forward(self, input_ids, attention_mask, labels=None):
    logits = self.student(input_ids, attention_mask)
    # rules = self.rule_evaluation.calculate_rules(logits)
    # # print("rules", rules)
    # regularized_student_logits = logits * rules
    # print("regularized_student shape", regularized_student_logits.shape)
    return torch.cat((logits, logits), dim=1)



In [16]:
class CustomLoss(nn.Module):
  def __init__(self):
    super(CustomLoss, self).__init__()
    self.BCE = nn.BCEWithLogitsLoss()
    self.KLDiv = nn.KLDivLoss(reduction="batchmean", log_target=True)
    self.teacher_loss = 0
    self.student_loss = 0

  def get_loss(self):
    return self.teacher_loss, self.student_loss

  def forward(self, logits, labels):
    student_logits = logits[:, :22]
    student_logits_sigmoid = torch.sigmoid(logits[:, :22])
    regularized_logits = nn.Sigmoid()(logits[:, 22:])

    # print("logits")
    # print(student_logits)
    # print(regularized_logits)

    bceLoss = self.BCE(student_logits, labels.float())
    totalKldLoss = 0
    self.student_loss = bceLoss
    for i in range(student_logits.shape[0]):
      for j in range(22):
        temp = self.KLDiv(torch.tensor([student_logits_sigmoid[i, j], 1 - student_logits_sigmoid[i, j]]).log(), torch.tensor([regularized_logits[i, j], 1 - regularized_logits[i, j]]).log())
        # print("temp is", temp)
        # if temp != 0:
        #   print("temp", temp)
        #   # print("probs_logits", student_logits[i, j], regularized_logits[i, j])
        totalKldLoss += temp

    totalKldLoss = totalKldLoss / (22 * student_logits.shape[0])
    # kldLoss = self.KLDiv(student_logits, regularized_logits)
    self.teacher_loss = totalKldLoss
    # print("bceLoss", bceLoss, "kldLoss", totalKldLoss)
    return bceLoss + totalKldLoss

In [17]:
train_dataset, eval_dataset = random_split(total_dataset, [int(0.9 * len(total_dataset)), len(total_dataset) - int(0.9 * len(total_dataset))])

In [18]:
from sklearn.metrics import f1_score

def evaluate(model, dataset):
  model.eval()

  predicitons = []
  actual_np = []

  with torch.no_grad():
    for i in range(len(dataset)):
      prediction = model(dataset[i]['input_ids'].squeeze(1).to(device), dataset[i]['attention_mask'].squeeze(1).to(device))
      predicitons.append(prediction[:,:22])
      actual_np.append(dataset[i]['narr_labels'].numpy())

    predcition_tensor = torch.sigmoid(torch.stack(predicitons).squeeze(1))
    predicted_labels_np = torch.where(predcition_tensor > 0.5, 1, 0).cpu().numpy()
    actual_np = np.array(actual_np)

    print("prediction", predicted_labels_np.shape, "actual", actual_np.shape)

    f1_avg_score_micro = f1_score(actual_np, predicted_labels_np, average='micro')
    f1_avg_score_macro = f1_score(actual_np, predicted_labels_np, average='macro')
    f1_score_weighted = f1_score(actual_np, predicted_labels_np, average='weighted')
    f1_score_samples = f1_score(actual_np, predicted_labels_np, average='samples')

    return f1_avg_score_micro, f1_avg_score_macro, f1_score_weighted, f1_score_samples


In [19]:
custom_loss = CustomLoss().to(device)
model = TeacherStudentNarrativeClassifier(22).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

NameError: name 'TeacherStudentNarrativeClassifier' is not defined

In [ ]:
train_loss_history = []
teacher_loss_history = []
student_loss_history = []

eval_f1_history = []

for epoch in range(3):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, dominantLabels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1])

        loss.backward()
        optimizer.step()

    eval_f1_micro, eval_f1_macro, eval_f1_weighted = evaluate(model, eval_dataset)
    eval_f1_history.append([eval_f1_micro, eval_f1_macro, eval_f1_weighted])
    print("epoch", epoch, "eval f1 micro", eval_f1_micro, "eval f1 macro", eval_f1_macro, "eval f1 weighted", eval_f1_weighted)

epoch 0
epoch 0 step 0 total loss 0.6575765013694763
teacher loss tensor(0.) student loss tensor(0.6576, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 100 total loss 0.23074662685394287
teacher loss tensor(0.) student loss tensor(0.2307, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


KeyboardInterrupt: 

In [ ]:
temp1 = torch.tensor([[0, 3, 4], [5, 6, 7]])

temp2 = torch.tensor([[2, 3, 4], [5, 6, 7]])

torch.sigmoid(temp1.float())

tensor([[0.5000, 0.9526, 0.9820],
        [0.9933, 0.9975, 0.9991]])

In [26]:
class RuleSubNarrNarr:
  def __init__(self, regularization_terms, confidence):
    self.regularization_terms = regularization_terms
    self.confidence = confidence

  def calculateTotalRules(self, sub_narr_logits, narr_logits, urw_logit, cc_logit):
    updated_probs = torch.zeros_like(sub_narr_logits)

    updated_probs = updated_probs + self.calculate_narr_subnarr_rule(sub_narr_logits, narr_logits) * self.regularization_terms[0]
    updated_probs = updated_probs + self.calculate_urw_cc_rule(sub_narr_logits, urw_logit, cc_logit) * self.regularization_terms[1]

    return torch.exp( -1 * updated_probs * self.confidence)

  def calculate_narr_subnarr_rule(self, sub_narr_logits, narr_logits):
    updated_probs = torch.zeros_like(sub_narr_logits)

    for i in range(96):
      prob_index1 = torch.sigmoid(sub_narr_logits[:, i])
      #mapping between parent and label needed
      prod_narr = torch.sigmoid(narr_logits[:, label2Narr[i]])

      # print("prob_index1", prob_index1)
      # print("prob_index2", prob_index2)
      rule_eval = 1 - torch.minimum(1 - prob_index1 + prod_narr, torch.ones_like(prob_index1))
      # print("rule_eval", rule_eval.shape, "updated_probs", updated_probs.shape)
      updated_probs[:, i] = rule_eval

    return updated_probs

  def calculate_urw_cc_rule(self, sub_narr_logits, urw_logit, cc_logit):
    updated_probs = torch.zeros_like(sub_narr_logits)

    for i in range(48):
      prob_index1_urw = torch.sigmoid(sub_narr_logits[:, i])
      prob_urw = torch.sigmoid(urw_logit)

      rule_eval = 1 - torch.minimum(1 - prob_index1_urw + prob_urw, torch.ones_like(prob_index1_urw))

      updated_probs[:, i] = rule_eval

    for i in range(48, 96):
      prob_index1_cc = torch.sigmoid(sub_narr_logits[:, i])
      prob_cc = torch.sigmoid(cc_logit)

      rule_eval = 1 - torch.minimum(1 - prob_index1_cc + prob_cc, torch.ones_like(prob_index1_cc))

      updated_probs[:, i] = rule_eval

    return updated_probs

In [21]:
class GlobalMaxPooling1D(nn.Module):

  def __init__(self, data_format='channels_last'):
      super(GlobalMaxPooling1D, self).__init__()
      self.data_format = data_format
      self.step_axis = 1 if self.data_format == 'channels_last' else 2

  def forward(self, input):
      return torch.max(input, axis=self.step_axis).values

In [ ]:
class BERTNarrativeClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTNarrativeClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # sequence_output = outputs.pooler_output
        # sequence_output = self.dropout(sequence_output)
        # logits = self.classifier(sequence_output)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        return logits

In [ ]:
narrativeClassifier = BERTNarrativeClassifier(22)

narrativeClassifier.load_state_dict(torch.load('drive/MyDrive/SemEvalTaskData/model/baselineNarrativeClassifier5Epoch.pth'))

<ipython-input-36-a8dc2a418dff>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  narrativeClassifier.load_state_dict(torch.load('drive/MyDrive/SemEvalTaskData/model/baseline

<All keys matched successfully>

In [ ]:
#student network
class BERTSubNarrativeClassifierPretrained(nn.Module):
    def __init__(self):
        super(BERTSubNarrativeClassifierPretrained, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 96)
        self.dominantClassifier = narrativeClassifier
        self.urw_cc_classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.pooler = GlobalMaxPooling1D()


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.pooler(outputs.last_hidden_state)
        sequence_output = self.dropout(sequence_output)

        cls_output = outputs.last_hidden_state[:, 0, :]

        logits = self.classifier(sequence_output)

        return logits, self.dominantClassifier(input_ids=input_ids, attention_mask=attention_mask), self.urw_cc_classifier(cls_output)

In [ ]:
class TeacherStudentNarrativeClassifierNeuroSymPretrained(nn.Module):
  def __init__(self):
    super(TeacherStudentNarrativeClassifierNeuroSymPretrained, self).__init__()
    self.student = BERTSubNarrativeClassifierPretrained()
    self.rule_evaluation = RuleSubNarrNarr([100, 100], 100)

  def forward(self, input_ids, attention_mask, labels=None):
    logits, narrLogits, urw_cc_logits = self.student(input_ids, attention_mask)
    rules = self.rule_evaluation.calculateTotalRules(logits, narrLogits, urw_cc_logits[:, 0], urw_cc_logits[:, 1])
    # print("rules", rules
    # # print("rules", rules)
    regularized_student_logits = logits * rules
    # print("regularized_student shape", regularized_student_logits.shape)
    return torch.cat((logits, regularized_student_logits, narrLogits, urw_cc_logits), dim=1)



In [33]:
#student network
class BERTSubNarrativeClassifier(nn.Module):
    def __init__(self):
        super(BERTSubNarrativeClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 96)
        self.dominantClassifier = nn.Linear(self.bert.config.hidden_size, 22)
        self.urw_cc_classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.pooler = GlobalMaxPooling1D()


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.pooler(outputs.last_hidden_state)
        sequence_output = self.dropout(sequence_output)

        cls_output = outputs.last_hidden_state[:, 0, :]

        logits = self.classifier(sequence_output)

        return logits, self.dominantClassifier(cls_output), self.urw_cc_classifier(cls_output)

In [34]:
class TeacherStudentNarrativeClassifierNeuroSym(nn.Module):
  def __init__(self):
    super(TeacherStudentNarrativeClassifierNeuroSym, self).__init__()
    self.student = BERTSubNarrativeClassifier()
    self.rule_evaluation = RuleSubNarrNarr([100, 100], 100)

  def forward(self, input_ids, attention_mask, labels=None):
    logits, narrLogits, urw_cc_logits = self.student(input_ids, attention_mask)
    rules = self.rule_evaluation.calculateTotalRules(logits, narrLogits, urw_cc_logits[:, 0], urw_cc_logits[:, 1])
    # print("rules", rules
    # # print("rules", rules)
    regularized_student_logits = logits * rules
    # print("regularized_student shape", regularized_student_logits.shape)
    return torch.cat((logits, regularized_student_logits, narrLogits, urw_cc_logits), dim=1)



In [13]:
class CustomLossNeuroSym(nn.Module):
  def __init__(self):
    super(CustomLossNeuroSym, self).__init__()
    self.BCESubNarr = nn.BCEWithLogitsLoss()
    self.KLDiv = nn.KLDivLoss(reduction="batchmean", log_target=True)
    self.BCENarr = nn.BCEWithLogitsLoss()
    self.BCEUrwCC = nn.BCEWithLogitsLoss()
    self.teacher_loss = 0
    self.narr_loss = 0
    self.urw_cc_loss = 0
    self.student_loss = 0

  def get_loss(self):
    return self.teacher_loss, self.student_loss, self.narr_loss, self.urw_cc_loss

  def forward(self, logits, labels, narrLabels, urw_cc_label):
    student_logits = logits[:, :96]
    student_logits_sigmoid = torch.sigmoid(logits[:, :96])
    regularized_logits = torch.sigmoid(logits[:, 96:192])
    narr_logits = logits[:, 192:214]
    urw_cc_logits = logits[:, 214:]

    # print("logits")
    # print(student_logits)
    # print(regularized_logits)

    bceLoss = self.BCESubNarr(student_logits, labels.float())
    narrBCE = self.BCENarr(narr_logits, narrLabels.float())
    urwCCBCE = self.BCEUrwCC(urw_cc_logits, urw_cc_label.float())

    totalKldLoss = 0
    self.student_loss = bceLoss
    self.narr_loss = narrBCE
    self.urw_cc_loss = urwCCBCE
    for i in range(student_logits.shape[0]):
      for j in range(96):
        temp = self.KLDiv(torch.tensor([student_logits_sigmoid[i, j], 1 - student_logits_sigmoid[i, j]]).log(), torch.tensor([regularized_logits[i, j], 1 - regularized_logits[i, j]]).log())
        # print("temp is", temp)
        # if temp != 0:
        #   print("temp", temp)
        #   # print("probs_logits", student_logits[i, j], regularized_logits[i, j])
        totalKldLoss += temp

    totalKldLoss = totalKldLoss / (96 * student_logits.shape[0])
    # kldLoss = self.KLDiv(student_logits, regularized_logits)
    self.teacher_loss = totalKldLoss
    # print("bceLoss", bceLoss, "kldLoss", totalKldLoss, "narrBCELoss", narrBCE, "urwCCBCE", urwCCBCE)
    return bceLoss + totalKldLoss + narrBCE + urwCCBCE
    # return bceLoss + totalKldLoss

In [14]:
from operator import sub
from sklearn.metrics import f1_score

def evaluateModelOverall(model, dataset):
  model.eval()

  subNarrPredictions = []
  narrPredictions = []
  urw_cc_predictions = []

  actual_subNarr_np = []
  actual_narr_np = []
  actual_urw_cc_np = []

  with torch.no_grad():
    for i in range(len(dataset)):
      prediction = model(dataset[i]['input_ids'].squeeze(1).to(device), dataset[i]['attention_mask'].squeeze(1).to(device))
      subNarrPredictions.append(prediction[:,:96])
      narrPredictions.append(prediction[:,192:214])
      urw_cc_predictions.append(prediction[:,214:])

      actual_subNarr_np.append(dataset[i]['sub_narr_labels'].numpy())
      actual_narr_np.append(dataset[i]['narr_labels'].numpy())
      actual_urw_cc_np.append(dataset[i]['urw_cc_labels'].numpy())

    sub_narr_predcition_tensor = torch.sigmoid(torch.stack(subNarrPredictions).squeeze(1))
    sub_narr_predicted_labels_np = torch.where(sub_narr_predcition_tensor > 0.5, 1, 0).cpu().numpy()
    actual_subNarr_np = np.array(actual_subNarr_np)

    narr_predcition_tensor = torch.sigmoid(torch.stack(narrPredictions).squeeze(1))
    narr_predicted_labels_np = torch.where(narr_predcition_tensor > 0.5, 1, 0).cpu().numpy()
    actual_narr_np = np.array(actual_narr_np)

    urw_cc_predictions_tensor = torch.sigmoid(torch.stack(urw_cc_predictions).squeeze(1))
    urw_cc_predicted_labels_np = torch.where(urw_cc_predictions_tensor > 0.5, 1, 0).cpu().numpy()
    actual_urw_cc_np = np.array(actual_urw_cc_np)

    # print("prediction", sub_narr_predicted_labels_np.shape, "actual", actual_subNarr_np.shape)

    f1_avg_score_micro_sub_narr = f1_score(actual_subNarr_np, sub_narr_predicted_labels_np, average='micro')
    f1_avg_score_macro_sub_narr = f1_score(actual_subNarr_np, sub_narr_predicted_labels_np, average='macro')
    f1_score_weighted_sub_narr = f1_score(actual_subNarr_np, sub_narr_predicted_labels_np, average='weighted')
    f1_avg_score_samples_sub_narr = f1_score(actual_subNarr_np, sub_narr_predicted_labels_np, average='samples')

    f1_avg_score_micro_narr = f1_score(actual_narr_np, narr_predicted_labels_np, average='micro')
    f1_avg_score_macro_narr = f1_score(actual_narr_np, narr_predicted_labels_np, average='macro')
    f1_score_weighted_narr = f1_score(actual_narr_np, narr_predicted_labels_np, average='weighted')
    f1_avg_score_samples_samples = f1_score(actual_narr_np, narr_predicted_labels_np, average='samples')

    f1_avg_score_micro_urw_cc = f1_score(actual_urw_cc_np, urw_cc_predicted_labels_np, average='micro')
    f1_avg_score_macro_urw_cc = f1_score(actual_urw_cc_np, urw_cc_predicted_labels_np, average='macro')
    f1_score_weighted_urw_cc = f1_score(actual_urw_cc_np, urw_cc_predicted_labels_np, average='weighted')

    result = {
        'sub_narr': [f1_avg_score_micro_sub_narr, f1_avg_score_macro_sub_narr, f1_score_weighted_sub_narr, f1_avg_score_samples_sub_narr],
        'narr': [f1_avg_score_micro_narr, f1_avg_score_macro_narr, f1_score_weighted_narr, f1_avg_score_samples_samples],
        'urw_cc': [f1_avg_score_micro_urw_cc, f1_avg_score_macro_urw_cc, f1_score_weighted_urw_cc],
    }

    return result


In [ ]:
model

In [35]:
custom_loss = CustomLossNeuroSym().to(device)
model = TeacherStudentNarrativeClassifierNeuroSym().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [36]:
train_loss_history = []
teacher_loss_history = []
student_loss_history = []
narr_loss_history = []
urw_cc_loss_history = []

eval_f1_history = []

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 0
epoch 0 step 0 total loss 2.1976664066314697
teacher loss tensor(0.0179) student loss tensor(0.7919, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.7009, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.6870, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 100 total loss 1.336059331893921
teacher loss tensor(0.0560) student loss tensor(0.1797, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2280, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.8724, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 200 total loss 1.3061014413833618
teacher loss tensor(0.0159) student loss tensor(0.0995, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2655, device='cuda:0',
       grad_fn=<BinaryCrossEn

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 0 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 0 narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 0 urw_cc_f1 [0.6421052631578947, 0.3860759493670886, 0.5345666991236612]
epoch 1
epoch 1 step 0 total loss 0.7065410614013672
teacher loss tensor(0.0062) student loss tensor(0.1031, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2383, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.3589, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 100 total loss 0.8430620431900024
teacher loss tensor(0.0302) student loss tensor(0.0909, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2477, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.4743, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 200 total loss 1.18178391456604
teacher loss tensor(0.0024) student loss ten

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 1 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 1 narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 1 urw_cc_f1 [0.7357512953367875, 0.6771665187814255, 0.7202238755033785]
epoch 2
epoch 2 step 0 total loss 0.9990532398223877
teacher loss tensor(0.0042) student loss tensor(0.1262, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2840, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.5848, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 100 total loss 0.7083784341812134
teacher loss tensor(0.0107) student loss tensor(0.0659, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1781, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.4537, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 200 total loss 1.20856511592865
teacher loss tensor(0.0137) student loss ten

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 2 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 2 narr_f1 [0.26053639846743293, 0.07479472942170216, 0.21901771642214496, 0.1405753968253968]
epoch 2 urw_cc_f1 [0.7623318385650224, 0.689388356669123, 0.7371464202709597]
epoch 3
epoch 3 step 0 total loss 0.5294891595840454
teacher loss tensor(0.0087) student loss tensor(0.0830, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2117, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2260, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 100 total loss 0.9500306248664856
teacher loss tensor(0.0219) student loss tensor(0.1111, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2298, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.5873, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 200 total loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 3 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 3 narr_f1 [0.13100436681222707, 0.03880914407230197, 0.08705672226543369, 0.1111111111111111]
epoch 3 urw_cc_f1 [0.7772511848341233, 0.7685916919959473, 0.7784895955108723]
epoch 4
epoch 4 step 0 total loss 0.855358898639679
teacher loss tensor(0.0166) student loss tensor(0.0737, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1862, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.5789, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 100 total loss 1.0644772052764893
teacher loss tensor(0.0305) student loss tensor(0.1008, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2313, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.7019, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 200 total loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

In [ ]:
for epoch in range(5, 10):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 5
epoch 5 step 0 total loss 0.8333513736724854
teacher loss tensor(0.0129) student loss tensor(0.1424, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2845, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.3936, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 5 step 100 total loss 0.9238169193267822
teacher loss tensor(0.0688) student loss tensor(0.0634, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1746, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.6170, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 5 step 200 total loss 0.48402535915374756
teacher loss tensor(0.0439) student loss tensor(0.1371, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2412, device='cuda:0',
       grad_fn=<BinaryCross

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 5 sub_narr_f1 [0.08791208791208792, 0.011386639676113358, 0.05553703262681591, 0.05597222222222222]
epoch 5 narr_f1 [0.2824427480916031, 0.09459344753462401, 0.2390962722460839, 0.14591931216931217]
epoch 5 urw_cc_f1 [0.8303571428571429, 0.8090696587190276, 0.8303952242241162]
epoch 6
epoch 6 step 0 total loss 0.4567415118217468
teacher loss tensor(0.0249) student loss tensor(0.0678, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1614, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2027, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 6 step 100 total loss 0.4787006676197052
teacher loss tensor(0.1228) student loss tensor(0.0814, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1714, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1031, device='cuda:0',
       grad_fn=<Bi

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 6 sub_narr_f1 [0.0661764705882353, 0.007233796296296296, 0.046296296296296294, 0.04486111111111111]
epoch 6 narr_f1 [0.24313725490196078, 0.08400111726265988, 0.20974668833313592, 0.11258597883597883]
epoch 6 urw_cc_f1 [0.7777777777777778, 0.741871921182266, 0.7752178855627132]
epoch 7
epoch 7 step 0 total loss 0.5278457403182983
teacher loss tensor(0.0630) student loss tensor(0.0742, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1903, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2005, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 7 step 100 total loss 0.5509144067764282
teacher loss tensor(0.0850) student loss tensor(0.0752, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2062, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1845, device='cuda:0',
       grad_fn=<B

In [ ]:
#Pretrained
custom_loss = CustomLossNeuroSym().to(device)
model = TeacherStudentNarrativeClassifierNeuroSymPretrained().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

In [ ]:
train_loss_history = []
teacher_loss_history = []
student_loss_history = []
narr_loss_history = []
urw_cc_loss_history = []

eval_f1_history = []

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 0
epoch 0 step 0 total loss 1.6787487268447876
teacher loss tensor(0.0422) student loss tensor(0.7862, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1028, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.7475, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 100 total loss 0.8652435541152954
teacher loss tensor(0.2442) student loss tensor(0.1601, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1661, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2949, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 200 total loss 0.6694949865341187
teacher loss tensor(0.3449) student loss tensor(0.1158, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0647, device='cuda:0',
       grad_fn=<BinaryCrossE

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 0 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 0 narr_f1 [0.7474226804123711, 0.37217272597822276, 0.7070226372551954, 0.6819071669071668]
epoch 0 urw_cc_f1 [0.9252336448598131, 0.9172380608685191, 0.9249523292278705]
epoch 1
epoch 1 step 0 total loss 0.8094605207443237
teacher loss tensor(0.3162) student loss tensor(0.1176, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0927, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2829, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 100 total loss 0.5956012010574341
teacher loss tensor(0.3925) student loss tensor(0.0782, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0510, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0739, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 200 total loss 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 1 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 1 narr_f1 [0.623229461756374, 0.2898126719897055, 0.5771228219320927, 0.5954298941798941]
epoch 1 urw_cc_f1 [0.9320388349514563, 0.9266877637130801, 0.9317479216276057]
epoch 2
epoch 2 step 0 total loss 0.8617931604385376
teacher loss tensor(0.3899) student loss tensor(0.0950, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0722, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.3047, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 100 total loss 0.8201414346694946
teacher loss tensor(0.5004) student loss tensor(0.0658, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0405, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2135, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 200 total loss 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 2 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 2 narr_f1 [0.7703703703703704, 0.4602334345673819, 0.7492093395435817, 0.7139616402116401]
epoch 2 urw_cc_f1 [0.9215686274509803, 0.903137039075399, 0.9200891470544963]
epoch 3
epoch 3 step 0 total loss 0.8347696661949158
teacher loss tensor(0.4288) student loss tensor(0.0831, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0521, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2708, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 100 total loss 0.8024264574050903
teacher loss tensor(0.4818) student loss tensor(0.0780, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0891, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1535, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 200 total loss 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 3 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 3 narr_f1 [0.6894865525672371, 0.42230399592640544, 0.6794216089611352, 0.6435299422799422]
epoch 3 urw_cc_f1 [0.9170731707317074, 0.9316129032258065, 0.9172788246566592]
epoch 4
epoch 4 step 0 total loss 0.5901263952255249
teacher loss tensor(0.3593) student loss tensor(0.1069, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0700, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0539, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 100 total loss 0.6999410390853882
teacher loss tensor(0.4975) student loss tensor(0.0640, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0439, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0945, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 200 total loss 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

In [ ]:
for epoch in range(5, 10):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 5
epoch 5 step 0 total loss 0.6562073230743408
teacher loss tensor(0.4954) student loss tensor(0.0690, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0318, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0600, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 5 step 100 total loss 0.623796284198761
teacher loss tensor(0.5126) student loss tensor(0.0323, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0259, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0530, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 5 step 200 total loss 0.5133546590805054
teacher loss tensor(0.3965) student loss tensor(0.0765, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0340, device='cuda:0',
       grad_fn=<BinaryCrossEn

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 5 sub_narr_f1 [0.07612456747404844, 0.0131368407578085, 0.05855048177628823, 0.027900571650571648]
epoch 5 narr_f1 [0.7017543859649122, 0.46503387130580687, 0.6787528724081587, 0.6782539682539682]
epoch 5 urw_cc_f1 [0.912621359223301, 0.8817934782608696, 0.9099225139905294]
epoch 6
epoch 6 step 0 total loss 0.8055500984191895
teacher loss tensor(0.4714) student loss tensor(0.0615, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0208, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2519, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 6 step 100 total loss 0.6507596969604492
teacher loss tensor(0.4805) student loss tensor(0.1000, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0626, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0076, device='cuda:0',
       grad_fn=<Binar

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 6 sub_narr_f1 [0.12244897959183673, 0.018148220309629228, 0.09846135521657343, 0.09758394383394384]
epoch 6 narr_f1 [0.7493917274939172, 0.4781351826804769, 0.7234635151212344, 0.7214087301587301]
epoch 6 urw_cc_f1 [0.8942307692307693, 0.8698336017176597, 0.8919234918660948]
epoch 7
epoch 7 step 0 total loss 0.6388734579086304
teacher loss tensor(0.5046) student loss tensor(0.0814, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0301, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0228, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 7 step 100 total loss 0.604289174079895
teacher loss tensor(0.4592) student loss tensor(0.0823, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0580, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0048, device='cuda:0',
       grad_fn=<Binar

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 7 sub_narr_f1 [0.1488673139158576, 0.019345238095238096, 0.10454545454545454, 0.14807539682539683]
epoch 7 narr_f1 [0.7209876543209877, 0.4200766567097617, 0.6861332958682219, 0.6676875901875902]
epoch 7 urw_cc_f1 [0.88, 0.893970893970894, 0.8792737901648794]
epoch 8
epoch 8 step 0 total loss 0.7853403091430664
teacher loss tensor(0.5515) student loss tensor(0.0794, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0244, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1301, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 8 step 100 total loss 0.7304222583770752
teacher loss tensor(0.5445) student loss tensor(0.0937, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0865, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0057, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWi

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 8 sub_narr_f1 [0.18471337579617833, 0.02888584869431644, 0.12816087138667784, 0.16686424686424683]
epoch 8 narr_f1 [0.745679012345679, 0.5037428439427466, 0.7328629421902958, 0.7152910052910053]
epoch 8 urw_cc_f1 [0.919431279620853, 0.9148555051349415, 0.9194985894577039]
epoch 9
epoch 9 step 0 total loss 0.6516599059104919
teacher loss tensor(0.5118) student loss tensor(0.1066, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0233, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0099, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 9 step 100 total loss 0.5913398861885071
teacher loss tensor(0.4777) student loss tensor(0.0866, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.0183, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0087, device='cuda:0',
       grad_fn=<BinaryC

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

In [ ]:
train_loss_history = []
teacher_loss_history = []
student_loss_history = []
narr_loss_history = []
urw_cc_loss_history = []

eval_f1_history = []

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

In [ ]:
torch.save(model.state_dict(), "drive/MyDrive/SemEvalTaskData/model/narrSubNarrNeuroSym10epoch.pth")

In [ ]:
eval_f1_history

[{'sub_narr': [0.0, 0.0, 0.0],
  'narr': [0.18181818181818182, 0.03896103896103896, 0.08866995073891625],
  'urw_cc': [0.9523809523809523, 0.9545454545454546, 0.9504132231404957]},
 {'sub_narr': [0.0, 0.0, 0.0],
  'narr': [0.12903225806451613, 0.045454545454545456, 0.10344827586206896],
  'urw_cc': [0.9565217391304348, 0.9545454545454546, 0.9586776859504131]},
 {'sub_narr': [0.0, 0.0, 0.0],
  'narr': [0.46153846153846156, 0.1829004329004329, 0.3243021346469622],
  'urw_cc': [1.0, 1.0, 1.0]},
 {'sub_narr': [0.13636363636363635, 0.01736111111111111, 0.059829059829059825],
  'narr': [0.358974358974359, 0.14015151515151514, 0.22701149425287356],
  'urw_cc': [1.0, 1.0, 1.0]},
 {'sub_narr': [0.13636363636363635, 0.022569444444444444, 0.08547008547008546],
  'narr': [0.32432432432432434, 0.13257575757575757, 0.20977011494252873],
  'urw_cc': [1.0, 1.0, 1.0]},
 {'sub_narr': [0.25, 0.03125, 0.13675213675213674],
  'narr': [0.4, 0.20833333333333331, 0.2672413793103448],
  'urw_cc': [0.9523809523

In [ ]:
##This is for Narrative Classifier - With Rules for

In [ ]:
dominantId2Label

{0: 'URW: Blaming the war on others rather than the invader',
 1: 'URW: Discrediting Ukraine',
 2: 'URW: Russia is the Victim',
 3: 'URW: Praise of Russia',
 4: 'URW: Overpraising the West',
 5: 'URW: Speculating war outcomes',
 6: 'URW: Discrediting the West, Diplomacy',
 7: 'URW: Negative Consequences for the West',
 8: 'URW: Distrust towards Media',
 9: 'URW: Amplifying war-related fears',
 10: 'URW: Hidden plots by secret schemes of powerful groups',
 11: 'CC: Criticism of climate policies',
 12: 'CC: Criticism of institutions and authorities',
 13: 'CC: Climate change is beneficial',
 14: 'CC: Downplaying climate change',
 15: 'CC: Questioning the measurements and science',
 16: 'CC: Criticism of climate movement',
 17: 'CC: Controversy about green technologies',
 18: 'CC: Hidden plots by secret schemes of powerful groups',
 19: 'CC: Amplifying Climate Fears',
 20: 'CC: Green policies are geopolitical instruments',
 21: 'Other'}

In [ ]:
class RuleNarr:
  def __init__(self, regularization_terms, confidence):
    self.regularization_terms = regularization_terms
    self.confidence = confidence

  def calculateTotalRules(self, narr_logits, urw_logit, cc_logit):
    updated_probs = torch.zeros_like(narr_logits)

    # updated_probs = updated_probs + self.calculate_narr_subnarr_rule(sub_narr_logits, narr_logits) * self.regularization_terms[0]
    updated_probs = updated_probs + self.calculate_urw_cc_rule(narr_logits, urw_logit, cc_logit) * self.regularization_terms[0]

    return torch.exp( -1 * updated_probs * self.confidence)

  def calculate_urw_cc_rule(self, narr_logits, urw_logit, cc_logit):
    updated_probs = torch.zeros_like(narr_logits)

    for i in range(11):
      prob_index1_urw = torch.sigmoid(narr_logits[:, i])
      prob_urw = torch.sigmoid(urw_logit)

      rule_eval = 1 - torch.minimum(1 - prob_index1_urw + prob_urw, torch.ones_like(prob_index1_urw))

      updated_probs[:, i] = rule_eval

    for i in range(11, 21):
      prob_index1_cc = torch.sigmoid(narr_logits[:, i])
      prob_cc = torch.sigmoid(cc_logit)

      rule_eval = 1 - torch.minimum(1 - prob_index1_cc + prob_cc, torch.ones_like(prob_index1_cc))

      updated_probs[:, i] = rule_eval

    return updated_probs

In [ ]:
# #student network
# class BERTSubNarrativeClassifier(nn.Module):
#     def __init__(self):
#         super(BERTSubNarrativeClassifier, self).__init__()
#         self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
#         self.dropout = nn.Dropout(0.1)
#         self.classifier = nn.Linear(self.bert.config.hidden_size, 96)
#         self.dominantClassifier = nn.Linear(self.bert.config.hidden_size, 22)
#         self.urw_cc_classifier = nn.Linear(self.bert.config.hidden_size, 2)
#         self.pooler = GlobalMaxPooling1D()


#     def forward(self, input_ids, attention_mask, labels=None):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         sequence_output = self.pooler(outputs.last_hidden_state)
#         sequence_output = self.dropout(sequence_output)

#         cls_output = outputs.last_hidden_state[:, 0, :]

#         logits = self.classifier(sequence_output)

#         return logits, self.dominantClassifier(cls_output), self.urw_cc_classifier(cls_output)

In [ ]:
class BERTClassifierNarr(nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifierNarr, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.urw_cc_classifier = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # sequence_output = outputs.pooler_output
        # sequence_output = self.dropout(sequence_output)
        # logits = self.classifier(sequence_output)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        return logits, self.urw_cc_classifier(cls_output)

In [ ]:
class TeacherStudentDominantNarrativeClassifierNeuroSym(nn.Module):
  def __init__(self):
    super(TeacherStudentDominantNarrativeClassifierNeuroSym, self).__init__()
    self.student = BERTClassifierNarr(22)
    self.rule_evaluation = RuleNarr([100], 100)

  def forward(self, input_ids, attention_mask, labels=None):
    logits, urw_cc_logits = self.student(input_ids, attention_mask)
    rules = self.rule_evaluation.calculateTotalRules(logits, urw_cc_logits[:, 0], urw_cc_logits[:, 1])
    # print("rules", rules
    # # print("rules", rules)
    regularized_student_logits = logits * rules
    # print("regularized_student shape", regularized_student_logits.shape)
    return torch.cat((logits, regularized_student_logits, urw_cc_logits), dim=1)



In [ ]:
class CustomLossNeuroSymNarr(nn.Module):
  def __init__(self):
    super(CustomLossNeuroSymNarr, self).__init__()
    self.KLDiv = nn.KLDivLoss(reduction="batchmean", log_target=True)
    self.BCENarr = nn.BCEWithLogitsLoss()
    self.BCEUrwCC = nn.BCEWithLogitsLoss()
    self.teacher_loss = 0
    self.urw_cc_loss = 0
    self.student_loss = 0

  def get_loss(self):
    return self.teacher_loss, self.student_loss, self.urw_cc_loss

  def forward(self, logits, labels, urw_cc_label):
    student_logits = logits[:, :22]
    student_logits_sigmoid = torch.sigmoid(logits[:, :22])
    regularized_logits = torch.sigmoid(logits[:, 22:44])
    urw_cc_logits = logits[:, 44:]

    # print("logits")
    # print(student_logits)
    # print(regularized_logits)

    bceLoss = self.BCENarr(student_logits, labels.float())
    urwCCBCE = self.BCEUrwCC(urw_cc_logits, urw_cc_label.float())

    totalKldLoss = 0
    self.student_loss = bceLoss
    self.urw_cc_loss = urwCCBCE
    for i in range(student_logits.shape[0]):
      for j in range(22):
        temp = self.KLDiv(torch.tensor([student_logits_sigmoid[i, j], 1 - student_logits_sigmoid[i, j]]).log(), torch.tensor([regularized_logits[i, j], 1 - regularized_logits[i, j]]).log())
        # print("temp is", temp)
        # if temp != 0:
        #   print("temp", temp)
        #   # print("probs_logits", student_logits[i, j], regularized_logits[i, j])
        totalKldLoss += temp

    totalKldLoss = totalKldLoss / (22 * student_logits.shape[0])
    # kldLoss = self.KLDiv(student_logits, regularized_logits)
    self.teacher_loss = totalKldLoss
    # print("bceLoss", bceLoss, "kldLoss", totalKldLoss, "narrBCELoss", narrBCE, "urwCCBCE", urwCCBCE)
    return bceLoss + totalKldLoss + urwCCBCE
    # return bceLoss + totalKldLoss

In [ ]:
from operator import sub
from sklearn.metrics import f1_score

def evaluateModelOverallNarr(model, dataset):
  model.eval()

  narrPredictions = []
  urw_cc_predictions = []

  actual_narr_np = []
  actual_urw_cc_np = []

  with torch.no_grad():
    for i in range(len(dataset)):
      prediction = model(dataset[i]['input_ids'].squeeze(1).to(device), dataset[i]['attention_mask'].squeeze(1).to(device))
      narrPredictions.append(prediction[:,:22])
      urw_cc_predictions.append(prediction[:,44:])

      actual_narr_np.append(dataset[i]['narr_labels'].numpy())
      actual_urw_cc_np.append(dataset[i]['urw_cc_labels'].numpy())

    narr_predcition_tensor = torch.sigmoid(torch.stack(narrPredictions).squeeze(1))
    narr_predicted_labels_np = torch.where(narr_predcition_tensor > 0.5, 1, 0).cpu().numpy()
    actual_narr_np = np.array(actual_narr_np)

    urw_cc_predictions_tensor = torch.sigmoid(torch.stack(urw_cc_predictions).squeeze(1))
    urw_cc_predicted_labels_np = torch.where(urw_cc_predictions_tensor > 0.5, 1, 0).cpu().numpy()
    actual_urw_cc_np = np.array(actual_urw_cc_np)

    # print("prediction", sub_narr_predicted_labels_np.shape, "actual", actual_subNarr_np.shape)

    f1_avg_score_micro_narr = f1_score(actual_narr_np, narr_predicted_labels_np, average='micro')
    f1_avg_score_macro_narr = f1_score(actual_narr_np, narr_predicted_labels_np, average='macro')
    f1_score_weighted_narr = f1_score(actual_narr_np, narr_predicted_labels_np, average='weighted')

    f1_avg_score_micro_urw_cc = f1_score(actual_urw_cc_np, urw_cc_predicted_labels_np, average='micro')
    f1_avg_score_macro_urw_cc = f1_score(actual_urw_cc_np, urw_cc_predicted_labels_np, average='macro')
    f1_score_weighted_urw_cc = f1_score(actual_urw_cc_np, urw_cc_predicted_labels_np, average='weighted')

    result = {
        'narr': [f1_avg_score_micro_narr, f1_avg_score_macro_narr, f1_score_weighted_narr],
        'urw_cc': [f1_avg_score_micro_urw_cc, f1_avg_score_macro_urw_cc, f1_score_weighted_urw_cc],
    }

    return result


In [ ]:
custom_loss = CustomLossNeuroSymNarr().to(device)
model = TeacherStudentDominantNarrativeClassifierNeuroSym().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

In [ ]:
train_loss_history = []
teacher_loss_history = []
student_loss_history = []
urw_cc_loss_history = []

eval_f1_history = []

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        urw_cc_loss_history.append(custom_loss.get_loss()[2])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "urw cc loss", custom_loss.get_loss()[2])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 0
epoch 0 step 0 total loss 1.3771605491638184
teacher loss tensor(0.0026) student loss tensor(0.6750, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.6995, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 100 total loss 0.4814952611923218
teacher loss tensor(0.1246) student loss tensor(0.2644, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0925, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 200 total loss 0.73179030418396
teacher loss tensor(0.1587) student loss tensor(0.2760, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2971, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0 narr_f1 [0.06451612903225806, 0.022727272727272728, 0.034482758620689655]
epoch 0 urw_cc_f1 [0.8181818181818182, 0.8117647058823529, 0.8164705882352941]
epoch 1
epoch 1 step 0 total loss 0.4200466275215149
teacher loss tensor(0.1591) student loss tensor(0.1804, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0805, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 100 total loss 1.0104869604110718
teacher loss tensor(0.2021) student loss tensor(0.2160, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.5924, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 200 total loss 0.5059295296669006
teacher loss tensor(0.2579) student loss tensor(0.1782, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0698, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1 narr_f1 [0.2777777777777778, 0.09696969696969697, 0.17011494252873563]
epoch 1 urw_cc_f1 [0.9523809523809523, 0.9666666666666667, 0.9533333333333334]
epoch 2
epoch 2 step 0 total loss 0.5204852819442749
teacher loss tensor(0.2214) student loss tensor(0.1878, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1112, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 100 total loss 0.6773984432220459
teacher loss tensor(0.2525) student loss tensor(0.2317, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1932, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 200 total loss 0.8294618129730225
teacher loss tensor(0.2593) student loss tensor(0.1283, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.4418, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 2 narr_f1 [0.3157894736842105, 0.10173160173160173, 0.24548440065681443]
epoch 2 urw_cc_f1 [0.8571428571428571, 0.8375, 0.8525]
epoch 3
epoch 3 step 0 total loss 0.5236299633979797
teacher loss tensor(0.2848) student loss tensor(0.1585, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0804, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 100 total loss 0.4797057509422302
teacher loss tensor(0.3203) student loss tensor(0.1250, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0344, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 200 total loss 0.5252261161804199
teacher loss tensor(0.2969) student loss tensor(0.1481, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0802, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 3 narr_f1 [0.2222222222222222, 0.09090909090909091, 0.16091954022988503]
epoch 3 urw_cc_f1 [0.9473684210526315, 0.9, 0.9400000000000001]
epoch 4
epoch 4 step 0 total loss 0.7029012441635132
teacher loss tensor(0.3263) student loss tensor(0.1341, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2425, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 100 total loss 0.5978479981422424
teacher loss tensor(0.3572) student loss tensor(0.2127, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0279, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 200 total loss 0.5300318002700806
teacher loss tensor(0.3266) student loss tensor(0.1681, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0354, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 narr_f1 [0.3902

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
class CustomLossNeuroSymNarrWithoutURWCC(nn.Module):
  def __init__(self):
    super(CustomLossNeuroSymNarrWithoutURWCC, self).__init__()
    self.KLDiv = nn.KLDivLoss(reduction="batchmean", log_target=True)
    self.BCENarr = nn.BCEWithLogitsLoss()
    self.teacher_loss = 0
    self.student_loss = 0

  def get_loss(self):
    return self.teacher_loss, self.student_loss

  def forward(self, logits, labels, urw_cc_label):
    student_logits = logits[:, :22]
    student_logits_sigmoid = torch.sigmoid(logits[:, :22])
    regularized_logits = torch.sigmoid(logits[:, 22:44])

    # print("logits")
    # print(student_logits)
    # print(regularized_logits)

    bceLoss = self.BCENarr(student_logits, labels.float())

    totalKldLoss = 0
    self.student_loss = bceLoss
    for i in range(student_logits.shape[0]):
      for j in range(22):
        temp = self.KLDiv(torch.tensor([student_logits_sigmoid[i, j], 1 - student_logits_sigmoid[i, j]]).log(), torch.tensor([regularized_logits[i, j], 1 - regularized_logits[i, j]]).log())
        # print("temp is", temp)
        # if temp != 0:
        #   print("temp", temp)
        #   # print("probs_logits", student_logits[i, j], regularized_logits[i, j])
        totalKldLoss += temp

    totalKldLoss = totalKldLoss / (22 * student_logits.shape[0])
    # kldLoss = self.KLDiv(student_logits, regularized_logits)
    self.teacher_loss = totalKldLoss
    # print("bceLoss", bceLoss, "kldLoss", totalKldLoss, "narrBCELoss", narrBCE, "urwCCBCE", urwCCBCE)
    return bceLoss + totalKldLoss
    # return bceLoss + totalKldLoss

In [ ]:
custom_loss = CustomLossNeuroSymNarrWithoutURWCC().to(device)

In [ ]:
for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        # urw_cc_loss_history.append(custom_loss.get_loss()[2])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    # print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 0
epoch 0 step 0 total loss 0.4782395660877228
teacher loss tensor(0.3685) student loss tensor(0.1098, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 100 total loss 0.2974056303501129
teacher loss tensor(0.1344) student loss tensor(0.1630, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 200 total loss 0.26942384243011475
teacher loss tensor(0.0656) student loss tensor(0.2038, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 0 narr_f1 [0.43902439024390244, 0.15714285714285714, 0.3251231527093596]
epoch 1
epoch 1 step 0 total loss 0.2542514503002167
teacher loss tensor(0.0579) student loss tensor(0.1964, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 100 total loss 0.13910409808158875
teacher loss tensor(0.0412) student loss tensor(0.0979, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 200 total loss 0.17131781578063965
teacher loss tensor(0.0677) student loss tensor(0.1036, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 1 narr_f1 [0.3333333333333333, 0.11374458874458875, 0.2308702791461412]
epoch 2
epoch 2 step 0 total loss 0.13245068490505219
teacher loss tensor(0.0508) student loss tensor(0.0816, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 100 total loss 0.20249314606189728
teacher loss tensor(0.) student loss tensor(0.2025, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 200 total loss 0.08484304696321487
teacher loss tensor(0.0047) student loss tensor(0.0801, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 2 narr_f1 [0.3902439024390244, 0.14494949494949497, 0.24674329501915707]
epoch 3
epoch 3 step 0 total loss 0.17858947813510895
teacher loss tensor(0.0036) student loss tensor(0.1750, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 100 total loss 0.09917002171278
teacher loss tensor(0.) student loss tensor(0.0992, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 200 total loss 0.10768534988164902
teacher loss tensor(0.0016) student loss tensor(0.1061, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch 3 narr_f1 [0.45454545454545453, 0.1525252525252525, 0.3386973180076628]
epoch 4
epoch 4 step 0 total loss 0.028384223580360413
teacher loss tensor(0.0025) student loss tensor(0.0259, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 100 total loss 0.09652413427829742
teacher loss tensor(0.0036) student loss tensor(0.0930, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 200 total loss 0.1085415855050087
teacher loss tensor(0.0007) student loss tensor(0.1078, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 narr_f1 [0.5116279069767442, 0.18434343434343434, 0.37547892720306514]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
class NewsDatasetMultilingual(Dataset):
  def __init__(self):
    self.files = files
    # self.annotation_file = annotations_file
    self.enSubTask2File = enSubTask2File
    self.hiSubTask2File = hiSubTask2File
    self.bgSubTask2File = bgSubTask2File
    self.ptSubTask2File = ptSubTask2File
    self.ruSubTask2File = ruSubTask2File
    self.tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index: int):
    file_name  = self.files[index]
    text = open(file_name, "r", encoding='utf-8').read()
    text_labels = []
    narr_text_labels = []
    total_labels = [0 for i in range(96)]
    narr_labels = [0 for i in range(22)]

    sub_narrative_df = {}
    narrative_df = {}

    for k in label2Id.keys():
      sub_narrative_df[k] = 0

    for k in dominantLabel2Id.keys():
      narrative_df[k] = 0

    encoded_input = self.tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

    if "EN" in file_name:
      annotation_file = self.enSubTask2File
    elif "HI" in file_name:
      annotation_file = self.hiSubTask2File
    elif "BG" in file_name:
      annotation_file = self.bgSubTask2File
    elif "PT" in file_name:
      annotation_file = self.ptSubTask2File
    elif "RU" in file_name:
      annotation_file = self.ruSubTask2File

    with open(annotation_file, encoding='utf-8') as annotations:
      for line in annotations:
        splits = line.split("\t")

        if file_name.split("/")[-1] == splits[0]:
          sub_narrative_df['fileName'] = splits[0]
          narrative_df['fileName'] = splits[0]
          text_labels = splits[2].replace("\n", "").split(';')
          narr_text_labels = splits[1].split(';')
          break

    is_CC = 0
    is_URW = 0

    for l in text_labels:
      total_labels[label2Id[l]] = 1
      sub_narrative_df[l] = 1

    for l in narr_text_labels:
      if dominantLabel2Id[l] <= 10:
        is_URW = 1
      elif dominantLabel2Id[l] <= 20:
        is_CC = 1

      narr_labels[dominantLabel2Id[l]] = 1
      narrative_df[l] = 1


    rec = {}

    rec = {
        **encoded_input,
        "narr_labels": torch.FloatTensor(narr_labels),
        "sub_narr_labels": torch.FloatTensor(total_labels),
        "narr_dic": narrative_df,
        "sub_narr_dic": sub_narrative_df,
        "fileName": file_name.split("/")[-1],
        "is_CC": is_CC,
        "is_URW": is_URW,
        "urw_cc_labels": torch.FloatTensor([is_URW, is_CC])
    }

    return rec

In [16]:
total_dataset = NewsDatasetMultilingual()
train_dataset, eval_dataset = random_split(total_dataset, [int(0.9 * len(total_dataset)), len(total_dataset) - int(0.9 * len(total_dataset))])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [27]:
#student network
class DeBERTaSubNarrativeClassifier(nn.Module):
    def __init__(self):
        super(DeBERTaSubNarrativeClassifier, self).__init__()
        self.bert = DebertaModel.from_pretrained('microsoft/deberta-base', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 96)
        self.dominantClassifier = nn.Linear(self.bert.config.hidden_size, 22)
        self.urw_cc_classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.pooler = GlobalMaxPooling1D()


    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.pooler(outputs.last_hidden_state)
        sequence_output = self.dropout(sequence_output)

        cls_output = outputs.last_hidden_state[:, 0, :]

        logits = self.classifier(sequence_output)

        return logits, self.dominantClassifier(cls_output), self.urw_cc_classifier(cls_output)

In [28]:
class TeacherStudentNarrativeClassifierNeuroSymDeBerta(nn.Module):
  def __init__(self):
    super(TeacherStudentNarrativeClassifierNeuroSymDeBerta, self).__init__()
    self.student = DeBERTaSubNarrativeClassifier()
    self.rule_evaluation = RuleSubNarrNarr([100, 100], 100)

  def forward(self, input_ids, attention_mask, labels=None):
    logits, narrLogits, urw_cc_logits = self.student(input_ids, attention_mask)
    rules = self.rule_evaluation.calculateTotalRules(logits, narrLogits, urw_cc_logits[:, 0], urw_cc_logits[:, 1])
    # print("rules", rules
    # # print("rules", rules)
    regularized_student_logits = logits * rules
    # print("regularized_student shape", regularized_student_logits.shape)
    return torch.cat((logits, regularized_student_logits, narrLogits, urw_cc_logits), dim=1)



In [29]:
custom_loss = CustomLossNeuroSym().to(device)
model = TeacherStudentNarrativeClassifierNeuroSymDeBerta().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
train_data_loader = DataLoader(train_dataset, shuffle=True, batch_size=5)

In [30]:
train_loss_history = []
teacher_loss_history = []
student_loss_history = []
narr_loss_history = []
urw_cc_loss_history = []

eval_f1_history = []

for epoch in range(5):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 0
epoch 0 step 0 total loss 2.387049436569214
teacher loss tensor(0.0526) student loss tensor(0.9322, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.6805, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.7217, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 100 total loss 1.3852970600128174
teacher loss tensor(0.0746) student loss tensor(0.1505, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.3419, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.8184, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 0 step 200 total loss 0.9406064748764038
teacher loss tensor(0.1386) student loss tensor(0.0723, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1794, device='cuda:0',
       grad_fn=<BinaryCrossEn

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 0 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 0 narr_f1 [0.1694915254237288, 0.03824833702882484, 0.1307220953982939, 0.07031746031746032]
epoch 0 urw_cc_f1 [0.7731958762886598, 0.7389277389277389, 0.7667204590281512]
epoch 1
epoch 1 step 0 total loss 0.8499906063079834
teacher loss tensor(0.0811) student loss tensor(0.1077, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2036, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.4576, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 100 total loss 0.8180272579193115
teacher loss tensor(0.1431) student loss tensor(0.0982, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2293, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.3474, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 1 step 200 total loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 1 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 1 narr_f1 [0.33098591549295775, 0.10747340675825155, 0.2879436501542825, 0.2272420634920635]
epoch 1 urw_cc_f1 [0.7374301675977654, 0.7278518518518519, 0.7371396011396011]
epoch 2
epoch 2 step 0 total loss 0.42858850955963135
teacher loss tensor(0.1079) student loss tensor(0.0807, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1657, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0743, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 100 total loss 0.7531699538230896
teacher loss tensor(0.1902) student loss tensor(0.1147, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2735, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1748, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 2 step 200 total los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 2 sub_narr_f1 [0.0, 0.0, 0.0, 0.0]
epoch 2 narr_f1 [0.1623931623931624, 0.05945328159404406, 0.12373242462230226, 0.13944444444444445]
epoch 2 urw_cc_f1 [0.7755102040816326, 0.7571428571428571, 0.7736263736263737]
epoch 3
epoch 3 step 0 total loss 0.5962709784507751
teacher loss tensor(0.2454) student loss tensor(0.1176, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2038, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0295, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 100 total loss 0.5458526611328125
teacher loss tensor(0.0949) student loss tensor(0.0770, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1578, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2161, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 3 step 200 total los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 3 sub_narr_f1 [0.023255813953488372, 0.003350389863547758, 0.02144249512670565, 0.01875]
epoch 3 narr_f1 [0.2222222222222222, 0.07431705793227993, 0.1660300998713188, 0.1813095238095238]
epoch 3 urw_cc_f1 [0.822429906542056, 0.7985993208828523, 0.8203931043489617]
epoch 4
epoch 4 step 0 total loss 0.5401514768600464
teacher loss tensor(0.1968) student loss tensor(0.0463, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1273, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1698, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 4 step 100 total loss 0.5208867192268372
teacher loss tensor(0.1496) student loss tensor(0.1072, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2183, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0458, device='cuda:0',
       grad_fn=<BinaryCrossEntr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

In [31]:
for epoch in range(5, 10):
    model.train()
    print("epoch", epoch)
    for i, data in enumerate(train_data_loader, 0):
        # print("epoch", epoch, "step", i)
        inputs = data
        input_ids = inputs['input_ids'].squeeze(1).to(device)
        attention_mask = inputs['attention_mask'].squeeze(1).to(device)
        dominantLabels = inputs['narr_labels'].to(device)
        subNarrLabels = inputs['sub_narr_labels'].to(device)
        urw_cc_labels = inputs['urw_cc_labels'].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # print("shape of input", inputs['input_ids'].shape, "max", torch.max(inputs['input_ids'], dim=0))
        # forward + backward + optimize
        # print("input_ids", input_ids.shape, "dominantLabels", dominantLabels.shape)
        outputs = model(input_ids, attention_mask).to(device)
        # print("shape of output", outputs.shape, "labels", labels.shape)
        loss = custom_loss(outputs, subNarrLabels, dominantLabels, urw_cc_labels)

        teacher_loss_history.append(custom_loss.get_loss()[0])
        student_loss_history.append(custom_loss.get_loss()[1])
        narr_loss_history.append(custom_loss.get_loss()[2])
        urw_cc_loss_history.append(custom_loss.get_loss()[3])

        train_loss_history.append(loss.item())
        if i % 100 == 0:
          print("epoch", epoch, "step", i, "total loss", loss.item())
          print("teacher loss", custom_loss.get_loss()[0], "student loss", custom_loss.get_loss()[1], "narr loss", custom_loss.get_loss()[2], "urw cc loss", custom_loss.get_loss()[3])

        loss.backward()
        optimizer.step()

    eval_result = evaluateModelOverall(model, eval_dataset)
    eval_f1_history.append(eval_result)
    print("epoch", epoch, "sub_narr_f1", eval_result['sub_narr'])
    print("epoch", epoch, "narr_f1", eval_result['narr'])
    print("epoch", epoch, "urw_cc_f1", eval_result['urw_cc'])

epoch 5
epoch 5 step 0 total loss 0.7369726896286011
teacher loss tensor(0.1132) student loss tensor(0.0814, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2108, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.3316, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 5 step 100 total loss 0.5488911867141724
teacher loss tensor(0.3215) student loss tensor(0.0870, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1161, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0242, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 5 step 200 total loss 0.9421870112419128
teacher loss tensor(0.1508) student loss tensor(0.0950, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1988, device='cuda:0',
       grad_fn=<BinaryCrossE

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 5 sub_narr_f1 [0.052434456928838954, 0.005787037037037038, 0.037037037037037035, 0.05277777777777778]
epoch 5 narr_f1 [0.26877470355731226, 0.082585660526837, 0.21523444626892901, 0.22250000000000003]
epoch 5 urw_cc_f1 [0.8, 0.7744090441932169, 0.7975334018499486]
epoch 6
epoch 6 step 0 total loss 0.6430712938308716
teacher loss tensor(0.2053) student loss tensor(0.0822, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2147, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1409, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 6 step 100 total loss 1.0172274112701416
teacher loss tensor(0.1952) student loss tensor(0.0643, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1613, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.5964, device='cuda:0',
       grad_fn=<BinaryCrossEntr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 6 sub_narr_f1 [0.09252669039145907, 0.011473429951690822, 0.06872406933788008, 0.07763888888888888]
epoch 6 narr_f1 [0.40924092409240925, 0.15950408706744443, 0.36437814373258415, 0.2796693121693122]
epoch 6 urw_cc_f1 [0.8516746411483254, 0.8298305084745763, 0.8491264667535854]
epoch 7
epoch 7 step 0 total loss 0.5116484761238098
teacher loss tensor(0.3259) student loss tensor(0.0608, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1174, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0076, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 7 step 100 total loss 0.44968098402023315
teacher loss tensor(0.2682) student loss tensor(0.0478, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1070, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0266, device='cuda:0',
       grad_fn=<

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 7 sub_narr_f1 [0.0989399293286219, 0.01103862597670647, 0.06649495711597332, 0.09569444444444446]
epoch 7 narr_f1 [0.2125984251968504, 0.06262520548234835, 0.16785344090059082, 0.1725]
epoch 7 urw_cc_f1 [0.7582417582417582, 0.667802108678021, 0.736614885519995]
epoch 8
epoch 8 step 0 total loss 0.7839246988296509
teacher loss tensor(0.3322) student loss tensor(0.0930, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.2107, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1480, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 8 step 100 total loss 0.5845867395401001
teacher loss tensor(0.1883) student loss tensor(0.0983, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1723, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.1257, device='cuda:0',
       grad_fn=<BinaryCrossEntropy

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

epoch 8 sub_narr_f1 [0.18421052631578946, 0.026447744829582138, 0.11669878984786995, 0.1496031746031746]
epoch 8 narr_f1 [0.37142857142857144, 0.14787639397908048, 0.3236949841704956, 0.29165343915343916]
epoch 8 urw_cc_f1 [0.8442211055276382, 0.8391690009337067, 0.8444659915248149]
epoch 9
epoch 9 step 0 total loss 0.5350846648216248
teacher loss tensor(0.2909) student loss tensor(0.0847, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1481, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.0114, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch 9 step 100 total loss 0.8228195905685425
teacher loss tensor(0.3080) student loss tensor(0.0677, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) narr loss tensor(0.1769, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) urw cc loss tensor(0.2702, device='cuda:0',
       grad_fn=<Bi

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

In [32]:
eval_result = evaluateModelOverall(model, total_dataset)
# eval_f1_history.append(eval_result)
print("sub_narr_f1", eval_result['sub_narr'])
print("narr_f1", eval_result['narr'])
print("urw_cc_f1", eval_result['urw_cc'])

sub_narr_f1 [0.20209059233449478, 0.02379338747681123, 0.1285932944745009, 0.18687367724867726]
narr_f1 [0.48002477547228245, 0.2214831648504605, 0.40185895598716104, 0.4039603174603174]
urw_cc_f1 [0.9542886492039034, 0.9569577893269109, 0.9543563576751783]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
